<a href="https://colab.research.google.com/github/Rutvikrj26/test/blob/main/PROD_BTP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install deepxde
!pip install matplotlib
!pip install tensorflow
!pip install numpy
!pip install scipy
!pip install tqdm
!pip install pickle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.3/192.3 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 10.7 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [2]:
"""
2-DOF Aeroelastic Flutter Simulation - Setup Module (Production Version)
------------------------------------------------------------------------
This module contains the enhanced setup for the aeroelastic flutter simulation.
"""

# Setup for DeepXDE with TensorFlow 1.x compatibility mode
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import jv, yv  # Bessel functions for Theodorsen

# Set DeepXDE backend before importing it
os.environ["DDE_BACKEND"] = "tensorflow.compat.v1"
import tensorflow as tf

# Critical: Disable eager execution for TensorFlow 1.x compatibility
tf.compat.v1.disable_eager_execution()

# Configure TensorFlow for better performance
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # Dynamically grow GPU memory allocation
config.log_device_placement = False     # Don't log device placement

# Set session configuration properly for TF 2.x compatibility
try:
    # For older TensorFlow versions
    tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))
except AttributeError:
    # For newer TensorFlow versions where set_session is not available
    tf.compat.v1.disable_eager_execution()
    tf.compat.v1.config.experimental.set_memory_growth(
        tf.config.experimental.list_physical_devices('GPU')[0], True
    ) if tf.config.experimental.list_physical_devices('GPU') else None

# Now import DeepXDE
import deepxde as dde

# Print versions and check eager execution
print(f"DeepXDE version: {dde.__version__}")
print(f"TensorFlow version: {tf.__version__}")
print(f"TensorFlow eager execution: {tf.executing_eagerly()}")

# Check GPU availability
if tf.test.is_gpu_available():
    print("GPU is available for training")
    gpu_device = tf.test.gpu_device_name()
    print(f"GPU device: {gpu_device}")
else:
    print("WARNING: No GPU detected, training will be slow on CPU")

# Function to test if DeepXDE is working properly
def test_deepxde_installation():
    """
    Test if DeepXDE is working correctly with a simple 1D problem
    Returns True if the test passes, False otherwise
    """
    print("Testing DeepXDE installation...")

    try:
        # Simple ODE test: u'' + u = 0, u(0) = 1, u'(0) = 0
        # Solution is u(x) = cos(x)
        def ode(x, y):
            dy_xx = dde.grad.hessian(y, x, i=0, j=0)
            return dy_xx + y

        # Domain
        geom = dde.geometry.TimeDomain(0, np.pi/2)

        # Boundary conditions
        def boundary_l(x, on_boundary):
            return on_boundary and np.isclose(x[0], 0)

        bc1 = dde.DirichletBC(geom, lambda x: 1, boundary_l)  # u(0) = 1
        bc2 = dde.NeumannBC(geom, lambda x: 0, boundary_l)    # u'(0) = 0

        # Create data
        data = dde.data.PDE(
            geom,
            ode,
            [bc1, bc2],
            num_domain=20,
            num_boundary=10,
            solution=lambda x: np.cos(x)  # Exact solution for verification
        )

        # Create a simple network for testing
        net = dde.nn.FNN([1, 20, 20, 1], "tanh", "Glorot uniform")

        # Create model
        model = dde.Model(data, net)

        # Compile and train briefly
        model.compile("adam", lr=0.001)
        model.train(iterations=100)

        # Predict and calculate error
        x = np.linspace(0, np.pi/2, 20)
        y_true = np.cos(x)
        y_pred = model.predict(x.reshape(-1, 1))

        # Compute error
        error = np.mean(np.abs(y_pred.flatten() - y_true))

        print(f"Test completed with mean error: {error:.6f}")
        return error < 0.1  # Tighter tolerance for production

    except Exception as e:
        print(f"DeepXDE test failed with error: {e}")
        import traceback
        traceback.print_exc()
        return False

# Adaptive sampling functions for better resolution
def adaptive_time_sampling(t_min, t_max, n_points):
    """
    Create adaptive time sampling with higher density near t=0

    Parameters:
    ----------
    t_min : float
        Start time
    t_max : float
        End time
    n_points : int
        Total number of points

    Returns:
    -------
    t : numpy.ndarray
        Time points with variable spacing
    """
    # Concentrate more points near t=0 where initial conditions apply
    # and dynamics are changing rapidly
    t_dense = np.linspace(t_min, t_min + 2.0, n_points // 3)
    t_medium = np.linspace(t_min + 2.0, t_min + 8.0, n_points // 3)
    t_sparse = np.linspace(t_min + 8.0, t_max, n_points // 3 + n_points % 3)
    return np.unique(np.concatenate([t_dense, t_medium, t_sparse]))

# Configure matplotlib for better plots
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman', 'DejaVu Serif'],
    'font.size': 12,
    'figure.figsize': [10, 8],
    'figure.dpi': 100,
    'axes.grid': True,
    'grid.alpha': 0.3,
    'lines.linewidth': 2
})

# Create directory for saving results
def create_results_directory(base_name="flutter_results"):
    """Create a directory for saving results with timestamp"""
    import time
    timestamp = time.strftime('%Y%m%d_%H%M%S')
    result_dir = f"{base_name}_{timestamp}"
    os.makedirs(result_dir, exist_ok=True)
    return result_dir

Using backend: tensorflow.compat.v1
Other supported backends: tensorflow, pytorch, jax, paddle.
paddle supports more examples now and is recommended.
Instructions for updating:
non-resource variables are not supported in the long term
Enable just-in-time compilation with XLA.

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


DeepXDE version: 1.13.2
TensorFlow version: 2.18.0
TensorFlow eager execution: False
GPU is available for training
GPU device: /device:GPU:0


In [3]:
"""
2-DOF Aeroelastic Flutter Simulation - Aeroelastic Parameters Module (Production Version)
-----------------------------------------------------------------------------------------
This module contains the enhanced AeroParams class for managing the physical parameters
of the aeroelastic system with full Theodorsen function implementation.
"""

import numpy as np
from scipy.special import jv, yv  # Bessel functions for Theodorsen

class AeroParams:
    """
    Class to store and calculate parameters for the 2-DoF aeroelastic flutter model.

    Parameters:
    ----------
    mu : float
        Mass ratio = m/(π*ρ∞*b²)
    r : float
        Radius of gyration = √(Ip/mb²)
    x_theta : float
        Normalized center of mass location
    a : float
        Normalized elastic axis location from mid-chord (-0.5 to 0.5)
    k_h : float
        Plunge spring stiffness (normalized)
    k_theta : float
        Pitch spring stiffness (normalized)
    name : str
        Identifier for this parameter set
    """
    def __init__(self, mu=20.0, r=0.5, x_theta=0.2, a=-0.4, k_h=1.0, k_theta=1.0, name=None):
        self.mu = mu
        self.r = r
        self.x_theta = x_theta
        self.a = a
        self.k_h = k_h
        self.k_theta = k_theta
        self.omega_h = np.sqrt(k_h)
        self.omega_theta = np.sqrt(k_theta)
        self.frequency_ratio = self.omega_h / self.omega_theta
        self.name = name or f"Params_mu{mu}_r{r}_a{a}"

        # Derived parameters for flutter analysis
        self.static_unbalance = x_theta / r**2
        self.radius_of_gyration_squared = r**2

    def compute_theodorsen(self, k):
        """
        Compute Theodorsen's function C(k) = F(k) + iG(k)
        using the exact formulation with Bessel functions

        Parameters:
        ----------
        k : float
            Reduced frequency

        Returns:
        -------
        F_k, G_k : float
            Real and imaginary parts of Theodorsen's function
        """
        if k < 1e-6:  # For very small k, return the limit value C(0) = 1
            return 1.0, 0.0

        # Compute using Bessel functions of first and second kind
        H2_1 = jv(1, k) - 1j * yv(1, k)
        H2_0 = jv(0, k) - 1j * yv(0, k)

        # Theodorsen function
        C_k = H2_1 / (H2_1 + 1j * H2_0)

        # Return real and imaginary parts
        return np.real(C_k), np.imag(C_k)

    def compute_aero_coefficients(self, velocity, omega=None):
        """
        Compute full unsteady aerodynamic coefficients with Theodorsen function

        Parameters:
        ----------
        velocity : float
            Dimensionless velocity U* = U/(b*ω_θ)
        omega : float or None
            Frequency of oscillation (default: use ω_θ)

        Returns:
        -------
        ell_h, ell_theta, m_h, m_theta : float
            Aerodynamic coefficients (real parts only for TensorFlow compatibility)
        """
        if omega is None:
            omega = self.omega_theta

        # Reduced frequency k = ω*b/U
        k = omega / velocity if velocity > 1e-6 else 1e6

        # Theodorsen function components
        F_k, G_k = self.compute_theodorsen(k)

        # Compute the circulatory (frequency-dependent) terms
        # Real parts of coefficients (in-phase with motion)
        ell_h_real = 2*np.pi * F_k  # Lift due to plunge
        ell_theta_real = 2*np.pi * (0.5 + self.a) * F_k  # Circulatory lift due to pitch

        m_h_real = 2*np.pi * (0.5 + self.a) * F_k  # Moment due to plunge
        m_theta_real = 2*np.pi * (0.5 + self.a)**2 * F_k  # Circulatory moment due to pitch

        # Add non-circulatory (added mass) terms
        ell_theta_real += np.pi/2  # Non-circulatory lift due to pitch
        m_theta_real += np.pi/8  # Non-circulatory moment due to pitch

        # For TensorFlow compatibility, we return only real parts
        # In a full complex implementation, both real and imaginary
        # parts would be used
        return ell_h_real, ell_theta_real, m_h_real, m_theta_real

    def estimate_flutter_speed(self):
        """
        Calculate an approximate analytical flutter speed
        for initialization purposes

        Returns:
        -------
        U_f : float
            Estimated flutter speed
        """
        # Using simplified formula based on equilibrium of
        # elastic and aerodynamic forces

        # Calculate static divergence speed (upper bound)
        U_div = np.sqrt(2 * self.mu * self.k_theta /
                        (np.pi * (0.5 + self.a)**2))

        # Flutter speed is typically lower than divergence speed
        # This is a rough approximation
        flutter_factor = 0.7 + 0.1 * self.r - 0.1 * abs(self.a)
        U_f = flutter_factor * U_div

        return U_f

    def compute_frequency_domain_matrices(self, velocity, reduced_freq):
        """
        Compute the frequency domain mass, damping, and stiffness matrices
        for flutter determinant analysis

        Parameters:
        ----------
        velocity : float
            Dimensionless velocity U* = U/(b*ω_θ)
        reduced_freq : float
            Reduced frequency k = ω*b/U

        Returns:
        -------
        M, C, K : numpy.ndarray
            Mass, damping, and stiffness matrices (2x2)
        """
        # Theodorsen function at this reduced frequency
        F_k, G_k = self.compute_theodorsen(reduced_freq)

        # Mass matrix
        M = np.array([
            [self.mu, self.mu * self.x_theta],
            [self.mu * self.x_theta, self.mu * self.r**2]
        ])

        # Damping matrix (includes aerodynamic damping)
        # Note: This is multiplied by p (complex frequency)
        C = np.array([
            [0, 0],
            [0, 0]
        ])
        if reduced_freq > 0:
            # Add aerodynamic damping terms (imaginary part of aero forces)
            # Scaled by velocity^2 / reduced_freq
            aero_factor = velocity**2 / reduced_freq
            C[0, 0] += aero_factor * 2*np.pi * G_k
            C[0, 1] += aero_factor * 2*np.pi * (0.5 + self.a) * G_k
            C[1, 0] += aero_factor * 2*np.pi * (0.5 + self.a) * G_k
            C[1, 1] += aero_factor * 2*np.pi * (0.5 + self.a)**2 * G_k

        # Stiffness matrix (includes aerodynamic stiffness)
        K = np.array([
            [self.k_h, 0],
            [0, self.k_theta]
        ])

        # Add aerodynamic stiffness terms (real part of aero forces)
        K[0, 0] += velocity**2 * 2*np.pi * F_k
        K[0, 1] += velocity**2 * (2*np.pi * (0.5 + self.a) * F_k + np.pi/2)
        K[1, 0] += velocity**2 * 2*np.pi * (0.5 + self.a) * F_k
        K[1, 1] += velocity**2 * (2*np.pi * (0.5 + self.a)**2 * F_k + np.pi/8)

        return M, C, K

    # Pre-defined parameter sets
    @classmethod
    def classical_section(cls):
        """Standard aeroelastic section model parameters"""
        return cls(
            mu=20.0,      # High mass ratio
            r=0.5,        # Standard radius of gyration
            x_theta=0.2,  # Mass center offset
            a=-0.4,       # Elastic axis behind mid-chord
            k_h=1.0,      # Equal natural frequencies
            k_theta=1.0,
            name="ClassicalSection"
        )

    @classmethod
    def light_aircraft(cls):
        """Light aircraft wing parameters"""
        return cls(
            mu=10.0,      # Lower mass ratio
            r=0.45,       # Different radius of gyration
            x_theta=0.1,  # Smaller offset
            a=-0.2,       # Elastic axis closer to mid-chord
            k_h=0.8,      # Frequency ratio ωh/ωθ = 0.8
            k_theta=1.25,
            name="LightAircraft"
        )

    @classmethod
    def non_conventional(cls):
        """Non-conventional aircraft parameters (e.g., high-altitude, composite)"""
        return cls(
            mu=5.0,       # Very low mass ratio
            r=0.6,        # Larger radius of gyration
            x_theta=0.3,  # Larger offset
            a=0.1,        # Elastic axis ahead of mid-chord (unusual)
            k_h=0.5,      # Very different natural frequencies
            k_theta=2.0,
            name="NonConventional"
        )

    def __str__(self):
        """Return a detailed string representation of the parameters"""
        return (f"AeroelasticParams '{self.name}':\n"
                f"  μ={self.mu:.2f} (mass ratio)\n"
                f"  r={self.r:.2f} (radius of gyration)\n"
                f"  xθ={self.x_theta:.2f} (center of mass)\n"
                f"  a={self.a:.2f} (elastic axis)\n"
                f"  kh={self.k_h:.2f}, kθ={self.k_theta:.2f} (stiffness)\n"
                f"  ωh/ωθ={self.frequency_ratio:.2f} (frequency ratio)\n"
                f"  Est. flutter speed U*≈{self.estimate_flutter_speed():.2f}")

In [4]:
"""
2-DOF Aeroelastic Flutter Simulation - Model Creation Module (Production Version)
--------------------------------------------------------------------------------
This module contains enhanced functions to create physics-informed neural network
models for the aeroelastic flutter simulation with deeper networks.
"""

import numpy as np
import deepxde as dde

def create_flutter_model(velocity, params, time_domain=(0, 30),
                         num_domain=5000, num_boundary=100, num_test=1000,
                         nn_type="fnn", nn_layers=None, activation="tanh"):
    """
    Create a production-quality PINN model for the 2-DoF aeroelastic flutter problem

    Parameters:
    ----------
    velocity : float
        Dimensionless flow velocity U* = U/(b*ω_θ)
    params : AeroParams
        Object containing system parameters
    time_domain : tuple
        (t_min, t_max) for the time domain
    num_domain : int
        Number of training points in the domain
    num_boundary : int
        Number of training points on the boundary
    num_test : int
        Number of test points
    nn_type : str
        Neural network type: "fnn" (feed-forward) or "resnet" (residual)
    nn_layers : list or None
        List of hidden layer sizes, or None for default
    activation : str
        Activation function: "tanh", "relu", "sigmoid", etc.

    Returns:
    -------
    model : dde.Model
        DeepXDE model for the flutter problem
    """
    # Create time domain
    geom = dde.geometry.TimeDomain(time_domain[0], time_domain[1])

    # Get aerodynamic coefficients with Theodorsen function
    ell_h, ell_theta, m_h, m_theta = params.compute_aero_coefficients(velocity)

    # Define the PDE system
    def pde(t, y):
        """
        Define the PDEs for the flutter system
        t: time variable
        y: [h, theta, h_dot, theta_dot]
        """
        # Extract solution components
        h = y[:, 0:1]         # Plunge displacement
        theta = y[:, 1:2]     # Pitch angle
        h_dot = y[:, 2:3]     # Plunge velocity
        theta_dot = y[:, 3:4] # Pitch velocity

        # Compute time derivatives using auto-differentiation
        h_t = dde.grad.jacobian(y, t, i=0)
        theta_t = dde.grad.jacobian(y, t, i=1)
        h_dot_t = dde.grad.jacobian(y, t, i=2)    # h_tt (acceleration)
        theta_dot_t = dde.grad.jacobian(y, t, i=3) # theta_tt (angular acceleration)

        # Equations of motion (from 5.62 in the textbook)
        # For plunge (h)
        eq1 = (params.mu * h_dot_t +
               params.mu * params.x_theta * theta_dot_t +
               params.k_h * h +
               velocity**2 * ell_h * h +
               velocity**2 * ell_theta * theta)

        # For pitch (theta)
        eq2 = (params.mu * params.x_theta * h_dot_t +
               params.mu * params.r**2 * theta_dot_t +
               params.k_theta * theta +
               velocity**2 * m_h * h +
               velocity**2 * m_theta * theta)

        # First-order system equations
        eq3 = h_dot - h_t
        eq4 = theta_dot - theta_t

        return [eq1, eq2, eq3, eq4]

    # Initial conditions at t=0
    def boundary_t0(x, on_boundary):
        return on_boundary and np.isclose(x[0], time_domain[0])

    # Set initial conditions with constant values
    bc1 = dde.DirichletBC(geom, lambda x: 0.01, boundary_t0, component=0)  # h(0) = 0.01
    bc2 = dde.DirichletBC(geom, lambda x: 0.0, boundary_t0, component=1)   # θ(0) = 0
    bc3 = dde.DirichletBC(geom, lambda x: 0.0, boundary_t0, component=2)   # h'(0) = 0
    bc4 = dde.DirichletBC(geom, lambda x: 0.0, boundary_t0, component=3)   # θ'(0) = 0

    # Create the data
    data = dde.data.PDE(
        geom,
        pde,
        [bc1, bc2, bc3, bc4],
        num_domain=num_domain,
        num_boundary=num_boundary,
        num_test=num_test,
        solution=None  # No analytical solution
    )

    # Default network architecture if not provided
    if nn_layers is None:
        nn_layers = [1, 64, 64, 64, 64, 4]  # Input dim=1, output dim=4
    else:
        # Make sure layers list has input and output dimensions
        if nn_layers[0] != 1:
            nn_layers = [1] + nn_layers
        if nn_layers[-1] != 4:
            nn_layers = nn_layers + [4]

    # Create appropriate neural network type
    if nn_type.lower() == "resnet":
        net = dde.nn.ResNet(
            nn_layers,
            activation,
            "Glorot normal"  # Better weight initialization for deep networks
        )
    else:  # Default to FNN
        net = dde.nn.FNN(
            nn_layers,
            activation,
            "Glorot normal"
        )

    # Create the model
    model = dde.Model(data, net)

    return model

def create_deep_flutter_model(velocity, params, time_domain=(0, 30), num_domain=10000):
    """
    Create a deep, production-quality PINN model

    Parameters:
    ----------
    velocity : float
        Dimensionless flow velocity U* = U/(b*ω_θ)
    params : AeroParams
        Object containing system parameters
    time_domain : tuple
        (t_min, t_max) for the time domain
    num_domain : int
        Number of training points in the domain

    Returns:
    -------
    model : dde.Model
        DeepXDE model with a deep architecture
    """
    # Deep network with 6 hidden layers of 64 neurons each
    nn_layers = [1, 64, 64, 64, 64, 64, 64, 4]

    return create_flutter_model(
        velocity=velocity,
        params=params,
        time_domain=time_domain,
        num_domain=num_domain,
        num_boundary=100,
        num_test=1000,
        nn_type="resnet",  # Use ResNet for better gradient flow
        nn_layers=nn_layers,
        activation="tanh"
    )

def create_hybrid_flutter_model(velocity, params, time_domain=(0, 30), num_domain=5000):
    """
    Create a hybrid model with physical components to speed up training

    Parameters:
    ----------
    velocity : float
        Dimensionless flow velocity U* = U/(b*ω_θ)
    params : AeroParams
        Object containing system parameters
    time_domain : tuple
        (t_min, t_max) for the time domain
    num_domain : int
        Number of training points in the domain

    Returns:
    -------
    model : dde.Model
        DeepXDE model with hybrid physics-ML architecture
    """
    # Create time domain
    geom = dde.geometry.TimeDomain(time_domain[0], time_domain[1])

    # Get aerodynamic coefficients
    ell_h, ell_theta, m_h, m_theta = params.compute_aero_coefficients(velocity)

    # Define the structured, physics-informed neural network
    def pde(t, y):
        """
        Define the PDEs with physical structure built-in
        """
        # Extract solution components
        h = y[:, 0:1]
        theta = y[:, 1:2]
        h_dot = y[:, 2:3]
        theta_dot = y[:, 3:4]

        # First-order equations (kinematic relationships)
        # These are enforced directly, not learned
        h_t = dde.grad.jacobian(y, t, i=0)
        theta_t = dde.grad.jacobian(y, t, i=1)
        eq3 = h_dot - h_t
        eq4 = theta_dot - theta_t

        # Compute accelerations
        h_tt = dde.grad.jacobian(y, t, i=2)
        theta_tt = dde.grad.jacobian(y, t, i=3)

        # Dynamic equations with physical structure enforced
        # Mass matrix terms
        M11 = params.mu
        M12 = params.mu * params.x_theta
        M21 = M12
        M22 = params.mu * params.r**2

        # Stiffness matrix terms (structural + aerodynamic)
        K11 = params.k_h + velocity**2 * ell_h
        K12 = velocity**2 * ell_theta
        K21 = velocity**2 * m_h
        K22 = params.k_theta + velocity**2 * m_theta

        # Equations of motion with matrix structure
        eq1 = M11 * h_tt + M12 * theta_tt + K11 * h + K12 * theta
        eq2 = M21 * h_tt + M22 * theta_tt + K21 * h + K22 * theta

        return [eq1, eq2, eq3, eq4]

    # Initial conditions
    def boundary_t0(x, on_boundary):
        return on_boundary and np.isclose(x[0], time_domain[0])

    # Set initial conditions
    bc1 = dde.DirichletBC(geom, lambda x: 0.01, boundary_t0, component=0)
    bc2 = dde.DirichletBC(geom, lambda x: 0.0, boundary_t0, component=1)
    bc3 = dde.DirichletBC(geom, lambda x: 0.0, boundary_t0, component=2)
    bc4 = dde.DirichletBC(geom, lambda x: 0.0, boundary_t0, component=3)

    # Create the data
    data = dde.data.PDE(
        geom,
        pde,
        [bc1, bc2, bc3, bc4],
        num_domain=num_domain,
        num_boundary=100,
        num_test=1000
    )

    # Define a network with skip connections to help with vanishing gradients
    net = dde.nn.PFNN(
        [1, 64, 64, 64, 64, 4],
        "tanh",
        "Glorot normal",
        regularization=["l2", 1e-5],  # L2 regularization for stability
        residual=True,  # Use residual connections
        trainable_stacked=True  # Allow training all layers
    )

    # Create the model
    model = dde.Model(data, net)

    return model

In [5]:
"""
2-DOF Aeroelastic Flutter Simulation - Training and Visualization Module (Production Version)
-------------------------------------------------------------------------------------------
This module contains enhanced functions for training the model and visualizing results.
"""

import os
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import tensorflow as tf

def train_production_model(model, total_iterations=10000, learning_rate=0.001,
                           checkpoint_freq=1000, result_dir=None):
    """
    Train the PINN model with professional training regime and checkpointing

    Parameters:
    ----------
    model : dde.Model
        DeepXDE model to train
    total_iterations : int
        Total number of training iterations
    learning_rate : float
        Initial learning rate
    checkpoint_freq : int
        Frequency of saving checkpoints
    result_dir : str or None
        Directory to save checkpoints and progress plots

    Returns:
    -------
    model : dde.Model
        Trained model
    losses : list
        Training losses
    checkpoint_dir : str
        Directory where checkpoints were saved
    """
    # Create checkpoint directory
    if result_dir is None:
        timestamp = time.strftime('%Y%m%d_%H%M%S')
        checkpoint_dir = f"flutter_checkpoints_{timestamp}"
    else:
        checkpoint_dir = os.path.join(result_dir, "checkpoints")

    os.makedirs(checkpoint_dir, exist_ok=True)

    # Initialize loss history
    all_losses = []

    # Phase 1: Initial training with higher learning rate
    print(f"Phase 1: Initial training with learning rate {learning_rate}")
    model.compile("adam", lr=learning_rate)

    # Train in smaller chunks with checkpoints
    iterations_completed = 0
    phase1_iterations = min(total_iterations // 2, 5000)

    while iterations_completed < phase1_iterations:
        batch_iterations = min(checkpoint_freq, phase1_iterations - iterations_completed)
        print(f"  Training iterations {iterations_completed + 1} to {iterations_completed + batch_iterations}...")

        try:
            losshistory, train_state = model.train(iterations=batch_iterations)
            current_losses = losshistory.loss_train
            all_losses.extend(current_losses)

            iterations_completed += batch_iterations

            # Save checkpoint
            checkpoint_file = os.path.join(checkpoint_dir, f"model_iter_{iterations_completed}.pkl")
            train_state.save(checkpoint_file)
            print(f"  Checkpoint saved to {checkpoint_file}")

            # Plot current loss history
            _plot_loss_history(all_losses, checkpoint_dir, iterations_completed)

            # Check for convergence (if loss is not decreasing much)
            if len(current_losses) > 10 and abs(current_losses[-1] - current_losses[0]) < 1e-4 * current_losses[0]:
                print("  Early convergence detected in Phase 1, moving to Phase 2...")
                break

        except Exception as e:
            print(f"  Warning: Error during Phase 1 training: {e}")
            print("  Continuing to Phase 2...")
            break

    # Phase 2: Fine-tuning with reduced learning rate
    print(f"Phase 2: Fine-tuning with reduced learning rate {learning_rate/10}")
    model.compile("adam", lr=learning_rate/10)

    phase2_iterations = min(total_iterations - iterations_completed, 3000)
    phase2_start = iterations_completed

    while iterations_completed < phase2_start + phase2_iterations:
        batch_iterations = min(checkpoint_freq, phase2_start + phase2_iterations - iterations_completed)
        print(f"  Training iterations {iterations_completed + 1} to {iterations_completed + batch_iterations}...")

        try:
            losshistory, train_state = model.train(iterations=batch_iterations)
            current_losses = losshistory.loss_train
            all_losses.extend(current_losses)

            iterations_completed += batch_iterations

            # Save checkpoint
            checkpoint_file = os.path.join(checkpoint_dir, f"model_iter_{iterations_completed}.pkl")
            train_state.save(checkpoint_file)
            print(f"  Checkpoint saved to {checkpoint_file}")

            # Plot current loss history
            _plot_loss_history(all_losses, checkpoint_dir, iterations_completed)

            # Check for convergence
            if len(current_losses) > 10 and abs(current_losses[-1] - current_losses[0]) < 1e-5 * current_losses[0]:
                print("  Early convergence detected in Phase 2, moving to final optimization...")
                break

        except Exception as e:
            print(f"  Warning: Error during Phase 2 training: {e}")
            print("  Continuing to final optimization...")
            break

    # Phase 3: Final optimization with L-BFGS (if available)
    print("Phase 3: Final optimization with L-BFGS")
    try:
        model.compile("L-BFGS")
        losshistory, train_state = model.train()

        # For L-BFGS, we might not get a full history, so just get the final loss
        if hasattr(losshistory, 'loss_train') and losshistory.loss_train:
            all_losses.extend(losshistory.loss_train)
        elif hasattr(losshistory, 'loss') and losshistory.loss:
            all_losses.append(losshistory.loss[-1])

        # Save final model
        checkpoint_file = os.path.join(checkpoint_dir, "model_final.pkl")
        train_state.save(checkpoint_file)
        print(f"  Final model saved to {checkpoint_file}")

        # Final loss plot
        _plot_loss_history(all_losses, checkpoint_dir, "final")

    except Exception as e:
        print(f"  Warning: L-BFGS optimization failed: {e}")
        print("  Using Adam-trained model as final result")

    print(f"Training completed with final loss: {all_losses[-1]:.10f}")
    return model, all_losses, checkpoint_dir

def _plot_loss_history(losses, save_dir, iteration):
    """Helper function to plot loss history"""
    plt.figure(figsize=(10, 6))
    plt.semilogy(losses)
    plt.title(f"Training Loss (through iteration {iteration})")
    plt.xlabel("Iterations")
    plt.ylabel("Loss (log scale)")
    plt.grid(True, which="both", ls="-")

    # Format y-axis to use scientific notation
    plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.1e'))

    save_path = os.path.join(save_dir, f"loss_iter_{iteration}.png")
    plt.savefig(save_path, dpi=150)
    plt.close()

def predict_and_visualize(model, time_domain=(0, 30), num_points=300,
                          velocity=None, result_dir=None):
    """
    Generate professional predictions and visualizations

    Parameters:
    ----------
    model : dde.Model
        Trained DeepXDE model
    time_domain : tuple
        (t_min, t_max) for prediction
    num_points : int
        Number of prediction points
    velocity : float or None
        Flow velocity for labeling (if known)
    result_dir : str or None
        Directory to save figures

    Returns:
    -------
    results : dict
        Dictionary with prediction results
    figs : list
        List of matplotlib figures
    """
    # Create adaptive time sampling for better resolution
    def adaptive_sampling(t_min, t_max, n_points):
        t_dense = np.linspace(t_min, t_min + (t_max-t_min)*0.2, n_points // 3)
        t_medium = np.linspace(t_min + (t_max-t_min)*0.2, t_min + (t_max-t_min)*0.6, n_points // 3)
        t_sparse = np.linspace(t_min + (t_max-t_min)*0.6, t_max, n_points // 3 + n_points % 3)
        return np.unique(np.concatenate([t_dense, t_medium, t_sparse]))

    # Create directory for saving results if not provided
    if result_dir is None:
        timestamp = time.strftime('%Y%m%d_%H%M%S')
        result_dir = f"flutter_results_{timestamp}"
        os.makedirs(result_dir, exist_ok=True)

    # Generate prediction points
    t_pred = adaptive_sampling(time_domain[0], time_domain[1], num_points)

    # Make predictions
    print(f"Generating predictions for time range {time_domain[0]} to {time_domain[1]}...")
    try:
        y_pred = model.predict(t_pred.reshape(-1, 1))

        # Extract solutions
        h_pred = y_pred[:, 0]       # Plunge
        theta_pred = y_pred[:, 1]   # Pitch
        h_dot_pred = y_pred[:, 2]   # Plunge velocity
        theta_dot_pred = y_pred[:, 3] # Pitch velocity

        # Store results
        results = {
            'time': t_pred,
            'h': h_pred,
            'theta': theta_pred,
            'h_dot': h_dot_pred,
            'theta_dot': theta_dot_pred
        }

        # Calculate amplitude growth (to detect flutter)
        mid_point = len(h_pred) // 2
        h_max_early = np.max(np.abs(h_pred[:mid_point]))
        h_max_late = np.max(np.abs(h_pred[mid_point:]))

        theta_max_early = np.max(np.abs(theta_pred[:mid_point]))
        theta_max_late = np.max(np.abs(theta_pred[mid_point:]))

        growth_ratio_h = h_max_late / h_max_early if h_max_early > 0 else 1.0
        growth_ratio_theta = theta_max_late / theta_max_early if theta_max_early > 0 else 1.0

        # Determine if flutter is occurring (simple criterion)
        is_flutter = (growth_ratio_h > 1.2) or (growth_ratio_theta > 1.2)

        # Add flutter metrics to results
        results['growth_ratio_h'] = growth_ratio_h
        results['growth_ratio_theta'] = growth_ratio_theta
        results['is_flutter'] = is_flutter

        # Print flutter metrics
        vel_str = f" at U* = {velocity:.2f}" if velocity is not None else ""
        print(f"Flutter analysis{vel_str}:")
        print(f"  Plunge growth ratio: {growth_ratio_h:.4f}")
        print(f"  Pitch growth ratio: {growth_ratio_theta:.4f}")
        print(f"  Flutter detected: {is_flutter}")

        # Create visualization figures
        figs = []

        # Figure 1: Time history of motion
        fig1, axs = plt.subplots(2, 2, figsize=(12, 8))

        # Plot plunge displacement
        axs[0, 0].plot(t_pred, h_pred, 'b-', linewidth=2)
        title = f"Plunge Displacement{vel_str}"
        if is_flutter:
            title += " (Flutter Detected)"
        axs[0, 0].set_title(title)
        axs[0, 0].set_xlabel("Time")
        axs[0, 0].set_ylabel("h/b")
        axs[0, 0].grid(True)

        # Plot pitch angle
        axs[0, 1].plot(t_pred, theta_pred, 'r-', linewidth=2)
        axs[0, 1].set_title(f"Pitch Angle{vel_str}")
        axs[0, 1].set_xlabel("Time")
        axs[0, 1].set_ylabel(r"$\theta$ (rad)")
        axs[0, 1].grid(True)

        # Plot plunge velocity
        axs[1, 0].plot(t_pred, h_dot_pred, 'b--', linewidth=2)
        axs[1, 0].set_title(f"Plunge Velocity{vel_str}")
        axs[1, 0].set_xlabel("Time")
        axs[1, 0].set_ylabel(r"$\dot{h}/b$")
        axs[1, 0].grid(True)

        # Plot pitch velocity
        axs[1, 1].plot(t_pred, theta_dot_pred, 'r--', linewidth=2)
        axs[1, 1].set_title(f"Pitch Angular Velocity{vel_str}")
        axs[1, 1].set_xlabel("Time")
        axs[1, 1].set_ylabel(r"$\dot{\theta}$ (rad/s)")
        axs[1, 1].grid(True)

        plt.tight_layout()
        fig1.savefig(os.path.join(result_dir, f"time_history{'_U{:.1f}'.format(velocity) if velocity is not None else ''}.png"), dpi=300)
        figs.append(fig1)

        # Figure 2: Phase portraits
        fig2, axs = plt.subplots(1, 2, figsize=(12, 5))

        # Phase portrait for plunge
        axs[0].plot(h_pred, h_dot_pred, 'b-', linewidth=1.5)
        axs[0].set_title(f"Plunge Phase Portrait{vel_str}")
        axs[0].set_xlabel("h/b")
        axs[0].set_ylabel(r"$\dot{h}/b$")
        axs[0].grid(True)

        # Phase portrait for pitch
        axs[1].plot(theta_pred, theta_dot_pred, 'r-', linewidth=1.5)
        axs[1].set_title(f"Pitch Phase Portrait{vel_str}")
        axs[1].set_xlabel(r"$\theta$ (rad)")
        axs[1].set_ylabel(r"$\dot{\theta}$ (rad/s)")
        axs[1].grid(True)

        plt.tight_layout()
        fig2.savefig(os.path.join(result_dir, f"phase_portrait{'_U{:.1f}'.format(velocity) if velocity is not None else ''}.png"), dpi=300)
        figs.append(fig2)

        # Figure 3: Coupled motion (h vs theta)
        fig3 = plt.figure(figsize=(8, 6))
        plt.plot(h_pred, theta_pred, 'g-', linewidth=1.5)
        plt.title(f"Coupled Plunge-Pitch Motion{vel_str}")
        plt.xlabel("h/b")
        plt.ylabel(r"$\theta$ (rad)")
        plt.grid(True)

        plt.tight_layout()
        fig3.savefig(os.path.join(result_dir, f"coupled_motion{'_U{:.1f}'.format(velocity) if velocity is not None else ''}.png"), dpi=300)
        figs.append(fig3)

        # Figure 4: Envelope analysis for flutter detection
        fig4, axs = plt.subplots(2, 1, figsize=(10, 8))

        # Calculate envelope using peaks
        from scipy.signal import find_peaks

        # Plunge envelope
        h_abs = np.abs(h_pred)
        h_peaks, _ = find_peaks(h_abs, height=0.1*np.max(h_abs))
        if len(h_peaks) > 0:
            axs[0].plot(t_pred, h_pred, 'b-', alpha=0.7, label='Plunge')
            axs[0].plot(t_pred[h_peaks], h_abs[h_peaks], 'ro', label='Peaks')

            # Fit exponential growth/decay to peaks if we have enough points
            if len(h_peaks) >= 5:
                from scipy.optimize import curve_fit

                def exp_func(x, a, b, c):
                    return a * np.exp(b * x) + c

                try:
                    popt, _ = curve_fit(exp_func, t_pred[h_peaks], h_abs[h_peaks],
                                      bounds=([0, -10, -10], [10, 10, 10]))

                    t_fit = np.linspace(t_pred[0], t_pred[-1], 1000)
                    h_fit = exp_func(t_fit, *popt)

                    axs[0].plot(t_fit, h_fit, 'k--',
                               label=f'Fit: {popt[0]:.2f}·exp({popt[1]:.3f}·t)+{popt[2]:.2f}')

                    # Add growth rate to results
                    results['h_growth_rate'] = popt[1]
                except:
                    pass

        axs[0].set_title(f"Plunge Envelope Analysis (Growth Ratio: {growth_ratio_h:.3f})")
        axs[0].set_xlabel("Time")
        axs[0].set_ylabel("h/b")
        axs[0].grid(True)
        axs[0].legend()

        # Pitch envelope
        theta_abs = np.abs(theta_pred)
        theta_peaks, _ = find_peaks(theta_abs, height=0.1*np.max(theta_abs))
        if len(theta_peaks) > 0:
            axs[1].plot(t_pred, theta_pred, 'r-', alpha=0.7, label='Pitch')
            axs[1].plot(t_pred[theta_peaks], theta_abs[theta_peaks], 'bo', label='Peaks')

            # Fit exponential growth/decay to peaks
            if len(theta_peaks) >= 5:
                try:
                    popt, _ = curve_fit(exp_func, t_pred[theta_peaks], theta_abs[theta_peaks],
                                      bounds=([0, -10, -10], [10, 10, 10]))

                    t_fit = np.linspace(t_pred[0], t_pred[-1], 1000)
                    theta_fit = exp_func(t_fit, *popt)

                    axs[1].plot(t_fit, theta_fit, 'k--',
                               label=f'Fit: {popt[0]:.2f}·exp({popt[1]:.3f}·t)+{popt[2]:.2f}')

                    # Add growth rate to results
                    results['theta_growth_rate'] = popt[1]
                except:
                    pass

        axs[1].set_title(f"Pitch Envelope Analysis (Growth Ratio: {growth_ratio_theta:.3f})")
        axs[1].set_xlabel("Time")
        axs[1].set_ylabel(r"$\theta$ (rad)")
        axs[1].grid(True)
        axs[1].legend()

        plt.tight_layout()
        fig4.savefig(os.path.join(result_dir, f"envelope_analysis{'_U{:.1f}'.format(velocity) if velocity is not None else ''}.png"), dpi=300)
        figs.append(fig4)

        print(f"Visualizations saved to {result_dir}/")
        return results, figs

    except Exception as e:
        print(f"Error during prediction and visualization: {e}")
        import traceback
        traceback.print_exc()
        return None, None

In [6]:
"""
2-DOF Aeroelastic Flutter Simulation - Flutter Analysis Module (Production Version)
----------------------------------------------------------------------------------
This module contains functions for comprehensive flutter boundary analysis.
"""

import os
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import pickle

from flutter_model import create_flutter_model
from flutter_training import train_production_model, predict_and_visualize

def analyze_flutter_boundary(params, velocity_range, time_domain=(0, 30),
                             nn_layers=None, iterations=5000, result_dir=None):
    """
    Conduct a comprehensive flutter boundary analysis across multiple velocities

    Parameters:
    ----------
    params : AeroParams
        Object containing system parameters
    velocity_range : list or np.ndarray
        List of velocities to analyze
    time_domain : tuple
        (t_min, t_max) for the time domain
    nn_layers : list or None
        Neural network architecture (if None, uses default)
    iterations : int
        Number of training iterations per velocity
    result_dir : str or None
        Directory to save results

    Returns:
    -------
    results : dict
        Results dictionary with data for each velocity
    flutter_speed : float or None
        Estimated flutter speed if detected
    """
    # Create results directory if not provided
    if result_dir is None:
        timestamp = time.strftime('%Y%m%d_%H%M%S')
        result_dir = f"flutter_analysis_{timestamp}"

    os.makedirs(result_dir, exist_ok=True)

    # Prepare for analysis
    print(f"=== Flutter Boundary Analysis ===")
    print(f"Parameter set: {params.name}")
    print(f"Analyzing {len(velocity_range)} velocities from U* = {min(velocity_range):.2f} to {max(velocity_range):.2f}")
    print(f"Results will be saved to: {result_dir}")

    # Initialize results
    results = {}
    flutter_detected = False
    flutter_speed = None

    # Create a file to log progress
    log_file = os.path.join(result_dir, "analysis_log.txt")
    with open(log_file, 'w') as f:
        f.write(f"Flutter Analysis Log\n")
        f.write(f"Started: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Parameter set: {params.name}\n")
        f.write(f"Velocities: {velocity_range}\n\n")

    # Analyze each velocity
    for i, velocity in enumerate(velocity_range):
        # Convert numpy value to Python float to avoid formatting issues
        velocity = float(velocity)
        velocity_dir = os.path.join(result_dir, f"velocity_{velocity:.2f}")
        os.makedirs(velocity_dir, exist_ok=True)

        print(f"\n[{i+1}/{len(velocity_range)}] Analyzing velocity U* = {velocity:.2f}")

        # Log analysis start
        with open(log_file, 'a') as f:
            f.write(f"\nVelocity U* = {velocity:.2f} - started at {time.strftime('%H:%M:%S')}\n")

        try:
            # 1. Create model
            model = create_flutter_model(
                velocity=velocity,
                params=params,
                time_domain=time_domain,
                num_domain=min(5000, iterations//2),  # Scale domain points with iterations
                nn_layers=nn_layers
            )

            # 2. Train model
            trained_model, losses, checkpoint_dir = train_production_model(
                model=model,
                total_iterations=iterations,
                learning_rate=0.001,
                result_dir=velocity_dir
            )

            # 3. Generate predictions and visualizations
            pred_results, figs = predict_and_visualize(
                model=trained_model,
                time_domain=time_domain,
                velocity=velocity,
                result_dir=velocity_dir
            )

            # 4. Evaluate flutter status
            is_flutter = pred_results['is_flutter']
            growth_ratio_h = pred_results['growth_ratio_h']
            growth_ratio_theta = pred_results['growth_ratio_theta']

            # Log results
            with open(log_file, 'a') as f:
                f.write(f"  Completed at {time.strftime('%H:%M:%S')}\n")
                f.write(f"  Growth ratio (h): {growth_ratio_h:.4f}\n")
                f.write(f"  Growth ratio (θ): {growth_ratio_theta:.4f}\n")
                f.write(f"  Flutter detected: {is_flutter}\n")

            # 5. Store results
            results[velocity] = {
                'time': pred_results['time'],
                'h': pred_results['h'],
                'theta': pred_results['theta'],
                'h_dot': pred_results['h_dot'],
                'theta_dot': pred_results['theta_dot'],
                'is_flutter': is_flutter,
                'growth_ratio_h': growth_ratio_h,
                'growth_ratio_theta': growth_ratio_theta,
                'losses': losses,
                'checkpoint_dir': checkpoint_dir
            }

            # Add growth rates if available
            if 'h_growth_rate' in pred_results:
                results[velocity]['h_growth_rate'] = pred_results['h_growth_rate']
            if 'theta_growth_rate' in pred_results:
                results[velocity]['theta_growth_rate'] = pred_results['theta_growth_rate']

            # Identify flutter boundary (first velocity where flutter is detected)
            if is_flutter and not flutter_detected:
                flutter_detected = True
                flutter_speed = velocity
                print(f"Flutter detected at velocity U* = {velocity:.2f}")

                # Log flutter detection
                with open(log_file, 'a') as f:
                    f.write(f"  FLUTTER BOUNDARY DETECTED\n")

            # Save partial results after each velocity
            with open(os.path.join(result_dir, "partial_results.pkl"), 'wb') as f:
                pickle.dump({
                    'params': params,
                    'results': results,
                    'flutter_speed': flutter_speed,
                    'velocities_analyzed': velocity_range[:i+1]
                }, f)

        except Exception as e:
            print(f"Error analyzing velocity U* = {velocity:.2f}: {e}")

            # Log error
            with open(log_file, 'a') as f:
                f.write(f"  ERROR: {e}\n")

            # Continue with next velocity
            continue

    # Create overall analysis visualizations
    _create_flutter_boundary_plots(results, flutter_speed, params, result_dir)

    # Save final results
    with open(os.path.join(result_dir, "final_results.pkl"), 'wb') as f:
        pickle.dump({
            'params': params,
            'results': results,
            'flutter_speed': flutter_speed,
            'velocities_analyzed': velocity_range
        }, f)

    # Log completion
    with open(log_file, 'a') as f:
        f.write(f"\nAnalysis completed at {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
        if flutter_speed:
            f.write(f"Flutter speed: U* = {flutter_speed:.4f}\n")
        else:
            f.write(f"No flutter detected in the analyzed velocity range\n")

    # Final report
    if flutter_speed:
        print(f"\nFlutter analysis complete. Flutter speed: U* = {flutter_speed:.4f}")
    else:
        print("\nFlutter analysis complete. No flutter detected in the analyzed velocity range.")

    print(f"All results saved to: {result_dir}")

    return results, flutter_speed

def refine_flutter_boundary(params, initial_results, flutter_speed,
                           refinement_range=0.5, num_points=5,
                           time_domain=(0, 30), iterations=8000, result_dir=None):
    """
    Perform a refined analysis around the estimated flutter speed

    Parameters:
    ----------
    params : AeroParams
        Object containing system parameters
    initial_results : dict
        Results from the first flutter boundary analysis
    flutter_speed : float
        Estimated flutter speed from initial analysis
    refinement_range : float
        Range around flutter speed to refine (±)
    num_points : int
        Number of refinement points
    time_domain : tuple
        (t_min, t_max) for the time domain
    iterations : int
        Number of training iterations per velocity
    result_dir : str or None
        Parent directory to save results

    Returns:
    -------
    refined_results : dict
        Combined results dictionary including refined analysis
    refined_flutter_speed : float
        Refined estimate of flutter speed
    """
    if flutter_speed is None:
        print("Cannot refine flutter boundary - no flutter speed detected in initial analysis")
        return initial_results, None

    # Create results directory if not provided
    if result_dir is None:
        result_dir = "flutter_analysis_refined"

    refinement_dir = os.path.join(result_dir, "refinement")
    os.makedirs(refinement_dir, exist_ok=True)

    # Generate refined velocity range
    # Convert to Python float if it's a numpy value
    flutter_speed = float(flutter_speed)
    v_min = max(0.1, flutter_speed - refinement_range)
    v_max = flutter_speed + refinement_range
    refined_velocities = np.linspace(v_min, v_max, num_points)

    print(f"\n=== Refined Flutter Boundary Analysis ===")
    print(f"Initial flutter speed estimate: U* = {flutter_speed:.4f}")
    print(f"Refining with {num_points} points from U* = {v_min:.4f} to {v_max:.4f}")

    # Run refined analysis
    refined_results, refined_flutter_speed = analyze_flutter_boundary(
        params=params,
        velocity_range=refined_velocities,
        time_domain=time_domain,
        iterations=iterations,  # More iterations for better accuracy
        result_dir=refinement_dir
    )

    # Merge results
    combined_results = {**initial_results}
    for v, data in refined_results.items():
        combined_results[v] = data

    # If refined analysis didn't find a flutter speed but initial did
    if refined_flutter_speed is None and flutter_speed is not None:
        refined_flutter_speed = flutter_speed

    # Create integrated visualization
    all_velocities = sorted(combined_results.keys())
    _create_flutter_boundary_plots(combined_results, refined_flutter_speed, params, result_dir)

    # Save combined results
    with open(os.path.join(result_dir, "combined_results.pkl"), 'wb') as f:
        pickle.dump({
            'params': params,
            'results': combined_results,
            'initial_flutter_speed': flutter_speed,
            'refined_flutter_speed': refined_flutter_speed,
            'velocities_analyzed': all_velocities
        }, f)

    print(f"\nRefined flutter analysis complete.")
    print(f"Refined flutter speed: U* = {refined_flutter_speed:.4f}")
    print(f"All results saved to: {result_dir}")

    return combined_results, refined_flutter_speed

def perform_frequency_domain_analysis(params, velocity_range, result_dir=None):
    """
    Perform classical frequency domain flutter analysis for comparison

    Parameters:
    ----------
    params : AeroParams
        Object containing system parameters
    velocity_range : list or np.ndarray
        List of velocities to analyze
    result_dir : str or None
        Directory to save results

    Returns:
    -------
    fd_results : dict
        Results of frequency domain analysis
    fd_flutter_speed : float or None
        Flutter speed estimated from frequency domain analysis
    """
    # Create results directory if not provided
    if result_dir is None:
        timestamp = time.strftime('%Y%m%d_%H%M%S')
        result_dir = f"flutter_fd_analysis_{timestamp}"

    freq_dir = os.path.join(result_dir, "frequency_domain")
    os.makedirs(freq_dir, exist_ok=True)

    print(f"\n=== Frequency Domain Flutter Analysis ===")

    # Prepare analysis
    # Range of reduced frequencies to analyze
    k_range = np.logspace(-2, 1, 100)  # Logarithmic spacing from 0.01 to 10

    # Initialize results
    fd_results = {}

    # For each velocity, calculate eigenvalues
    for velocity in velocity_range:
        print(f"Analyzing velocity U* = {velocity:.2f}")

        # Initialize arrays for results
        eigenvalues = np.zeros((len(k_range), 4), dtype=complex)
        damping = np.zeros((len(k_range), 2))
        frequency = np.zeros((len(k_range), 2))

        # Analyze each reduced frequency
        for i, k in enumerate(k_range):
            # Compute mass, damping, and stiffness matrices
            M, C, K = params.compute_frequency_domain_matrices(velocity, k)

            # Formulate eigenvalue problem: [M*p^2 + C*p + K]{q} = {0}
            # Convert to standard form: [A]{x} = p{x}
            # where [A] = [  0    I  ]
            #              [-M^-1*K -M^-1*C]
            M_inv = np.linalg.inv(M)

            # State-space form
            A = np.zeros((4, 4))
            A[0:2, 2:4] = np.eye(2)
            A[2:4, 0:2] = -np.dot(M_inv, K)
            A[2:4, 2:4] = -np.dot(M_inv, C)

            # Compute eigenvalues
            eigvals = np.linalg.eigvals(A)
            eigenvalues[i, :] = eigvals

            # Extract natural frequency and damping from eigenvalues
            # Sort eigenvalues by imaginary part (frequency)
            sorted_idx = np.argsort(np.abs(np.imag(eigvals)))

            for j in range(2):
                p = eigvals[sorted_idx[j+2]]  # Skip conjugate pairs
                damping[i, j] = -np.real(p) / np.abs(p)
                frequency[i, j] = np.abs(np.imag(p))

        # Store results
        fd_results[velocity] = {
            'k_range': k_range,
            'eigenvalues': eigenvalues,
            'damping': damping,
            'frequency': frequency
        }

        # Check for flutter (damping becomes negative)
        for j in range(2):
            if np.any(damping[:, j] < 0):
                print(f"  Mode {j+1} shows negative damping - flutter detected")

    # Estimate flutter speed by interpolating damping values
    mode_1_damping = np.array([np.min(fd_results[v]['damping'][:, 0]) for v in velocity_range])
    mode_2_damping = np.array([np.min(fd_results[v]['damping'][:, 1]) for v in velocity_range])

    # Identify where damping becomes negative
    try:
        from scipy.interpolate import interp1d

        # Mode 1 flutter check
        if np.any(mode_1_damping < 0):
            # Find two points around zero damping
            v_below = np.max(velocity_range[mode_1_damping > 0])
            v_above = np.min(velocity_range[mode_1_damping < 0])

            # Interpolate
            v_interp = np.linspace(v_below, v_above, 100)
            damping_interp = interp1d(velocity_range, mode_1_damping)(v_interp)
            mode_1_flutter = v_interp[np.argmin(np.abs(damping_interp))]
        else:
            mode_1_flutter = None

        # Mode 2 flutter check
        if np.any(mode_2_damping < 0):
            # Find two points around zero damping
            v_below = np.max(velocity_range[mode_2_damping > 0])
            v_above = np.min(velocity_range[mode_2_damping < 0])

            # Interpolate
            v_interp = np.linspace(v_below, v_above, 100)
            damping_interp = interp1d(velocity_range, mode_2_damping)(v_interp)
            mode_2_flutter = v_interp[np.argmin(np.abs(damping_interp))]
        else:
            mode_2_flutter = None

        # Overall flutter speed is the lower of the two
        if mode_1_flutter and mode_2_flutter:
            fd_flutter_speed = min(mode_1_flutter, mode_2_flutter)
        elif mode_1_flutter:
            fd_flutter_speed = mode_1_flutter
        elif mode_2_flutter:
            fd_flutter_speed = mode_2_flutter
        else:
            fd_flutter_speed = None

    except Exception as e:
        print(f"Error in flutter speed estimation: {e}")
        fd_flutter_speed = None

    # Create plots
    _create_frequency_domain_plots(fd_results, velocity_range, fd_flutter_speed, freq_dir)

    # Save results
    with open(os.path.join(freq_dir, "fd_results.pkl"), 'wb') as f:
        pickle.dump({
            'params': params,
            'fd_results': fd_results,
            'fd_flutter_speed': fd_flutter_speed
        }, f)

    # Final report
    if fd_flutter_speed:
        print(f"\nFrequency domain analysis complete. Flutter speed: U* = {fd_flutter_speed:.4f}")
    else:
        print("\nFrequency domain analysis complete. No flutter detected.")

    return fd_results, fd_flutter_speed

def _create_flutter_boundary_plots(results, flutter_speed, params, result_dir):
    """Helper function to create flutter boundary analysis plots"""
    velocities = sorted(list(results.keys()))

    # 1. Growth ratio plot
    plt.figure(figsize=(10, 6))

    growth_ratios_h = [results[v]['growth_ratio_h'] for v in velocities]
    growth_ratios_theta = [results[v]['growth_ratio_theta'] for v in velocities]

    plt.plot(velocities, growth_ratios_h, 'bo-', linewidth=2, label="Plunge Growth Ratio")
    plt.plot(velocities, growth_ratios_theta, 'rs-', linewidth=2, label="Pitch Growth Ratio")

    # Flutter threshold line
    plt.axhline(y=1.0, color='k', linestyle='--', label="Stability Boundary")

    # Mark flutter speed if detected
    if flutter_speed is not None:
        # Convert to Python float if numpy value
        flutter_speed = float(flutter_speed)
        plt.axvline(x=flutter_speed, color='g', linestyle='-', linewidth=2,
                   label=f"Flutter Speed U* = {flutter_speed:.4f}")

    # Add annotations for stable/unstable regions
    if flutter_speed is not None:
        plt.text(flutter_speed*0.7, 0.7, "STABLE", fontsize=12, color='blue')
        plt.text(flutter_speed*1.1, 1.4, "UNSTABLE", fontsize=12, color='red')

    plt.xlabel("Reduced Velocity (U*)")
    plt.ylabel("Amplitude Growth Ratio")
    plt.title(f"Aeroelastic Stability Analysis - {params.name}")
    plt.grid(True)
    plt.legend()

    plt.tight_layout()
    plt.savefig(os.path.join(result_dir, "stability_analysis.png"), dpi=300)

    # 2. Growth rate plot (if available)
    if all('h_growth_rate' in results[v] for v in velocities):
        plt.figure(figsize=(10, 6))

        growth_rates_h = [results[v].get('h_growth_rate', 0) for v in velocities]
        growth_rates_theta = [results[v].get('theta_growth_rate', 0) for v in velocities]

        plt.plot(velocities, growth_rates_h, 'bo-', linewidth=2, label="Plunge Growth Rate")
        plt.plot(velocities, growth_rates_theta, 'rs-', linewidth=2, label="Pitch Growth Rate")

        # Zero line (stability boundary)
        plt.axhline(y=0, color='k', linestyle='--', label="Stability Boundary")

        # Mark flutter speed if detected
        if flutter_speed is not None:
            flutter_speed = float(flutter_speed)  # Ensure it's a Python float
            plt.axvline(x=flutter_speed, color='g', linestyle='-', linewidth=2,
                       label=f"Flutter Speed U* = {flutter_speed:.4f}")

        plt.xlabel("Reduced Velocity (U*)")
        plt.ylabel("Exponential Growth Rate")
        plt.title(f"Aeroelastic Growth Rate Analysis - {params.name}")
        plt.grid(True)
        plt.legend()

        plt.tight_layout()
        plt.savefig(os.path.join(result_dir, "growth_rate_analysis.png"), dpi=300)

    # 3. Final loss values
    plt.figure(figsize=(8, 6))

    final_losses = [results[v]['losses'][-1] if 'losses' in results[v] and results[v]['losses'] else np.nan
                    for v in velocities]

    plt.semilogy(velocities, final_losses, 'ko-', linewidth=2)

    # Mark flutter speed if detected
    if flutter_speed is not None:
        flutter_speed = float(flutter_speed)  # Ensure it's a Python float
        plt.axvline(x=flutter_speed, color='g', linestyle='--', linewidth=2,
                  label=f"Flutter Speed U* = {flutter_speed:.4f}")

    plt.xlabel("Reduced Velocity (U*)")
    plt.ylabel("Final Loss Value")
    plt.title("Training Convergence vs. Velocity")
    plt.grid(True, which="both")

    if flutter_speed is not None:
        plt.legend()

    plt.tight_layout()
    plt.savefig(os.path.join(result_dir, "convergence_analysis.png"), dpi=300)

    # 4. Time history comparison for selected velocities
    if len(velocities) >= 3:
        # Choose 3 representative velocities: below, near, and above flutter
        if flutter_speed is not None:
            flutter_speed = float(flutter_speed)  # Ensure it's a Python float
            v_below = max([v for v in velocities if v < flutter_speed])
            v_near = min([v for v in velocities if v >= flutter_speed])
            v_above = max([v for v in velocities if v > flutter_speed * 1.1]) if any(v > flutter_speed * 1.1 for v in velocities) else max(velocities)
            selected = [v_below, v_near, v_above]
        else:
            # If no flutter, pick first, middle, last
            idx = [0, len(velocities)//2, -1]
            selected = [velocities[i] for i in idx]

        fig, axs = plt.subplots(2, 3, figsize=(15, 8))

        for i, v in enumerate(selected):
            data = results[v]

            # Plunge plot
            axs[0, i].plot(data['time'], data['h'], 'b-', linewidth=1.5)
            axs[0, i].set_title(f"Plunge at U* = {float(v):.2f}")
            axs[0, i].set_xlabel("Time")
            axs[0, i].set_ylabel("h/b")
            axs[0, i].grid(True)

            # Pitch plot
            axs[1, i].plot(data['time'], data['theta'], 'r-', linewidth=1.5)
            axs[1, i].set_title(f"Pitch at U* = {float(v):.2f}")
            axs[1, i].set_xlabel("Time")
            axs[1, i].set_ylabel(r"$\theta$ (rad)")
            axs[1, i].grid(True)

        plt.tight_layout()
        plt.savefig(os.path.join(result_dir, "velocity_comparison.png"), dpi=300)

    # 5. Phase portrait comparison
    if len(velocities) >= 3:
        if flutter_speed is not None:
            flutter_speed = float(flutter_speed)  # Ensure it's a Python float
            v_below = max([v for v in velocities if v < flutter_speed])
            v_near = min([v for v in velocities if v >= flutter_speed])
            v_above = max([v for v in velocities if v > flutter_speed * 1.1]) if any(v > flutter_speed * 1.1 for v in velocities) else max(velocities)
            selected = [v_below, v_near, v_above]
        else:
            # If no flutter, pick first, middle, last
            idx = [0, len(velocities)//2, -1]
            selected = [velocities[i] for i in idx]

        fig, axs = plt.subplots(1, 3, figsize=(15, 5))

        for i, v in enumerate(selected):
            data = results[v]

            # Phase portrait
            axs[i].plot(data['h'], data['theta'], 'g-', linewidth=1)
            axs[i].set_title(f"Phase Portrait at U* = {float(v):.2f}")
            axs[i].set_xlabel("h/b")
            axs[i].set_ylabel(r"$\theta$ (rad)")
            axs[i].grid(True)

        plt.tight_layout()
        plt.savefig(os.path.join(result_dir, "phase_comparison.png"), dpi=300)

def _create_frequency_domain_plots(fd_results, velocity_range, fd_flutter_speed, result_dir):
    """Helper function to create frequency domain analysis plots"""
    # Ensure velocity range contains Python floats, not numpy values
    velocity_range = [float(v) for v in velocity_range]

    # 1. Damping vs. velocity plot
    plt.figure(figsize=(10, 6))

    mode_1_damping = np.array([np.min(fd_results[v]['damping'][:, 0]) for v in velocity_range])
    mode_2_damping = np.array([np.min(fd_results[v]['damping'][:, 1]) for v in velocity_range])

    plt.plot(velocity_range, mode_1_damping, 'bo-', linewidth=2, label="Mode 1 Damping")
    plt.plot(velocity_range, mode_2_damping, 'rs-', linewidth=2, label="Mode 2 Damping")

    # Zero damping line (stability boundary)
    plt.axhline(y=0, color='k', linestyle='--', label="Stability Boundary")

    # Mark flutter speed if detected
    if fd_flutter_speed is not None:
        # Make sure it's a Python float
        fd_flutter_speed = float(fd_flutter_speed)
        plt.axvline(x=fd_flutter_speed, color='g', linestyle='-', linewidth=2,
                   label=f"Flutter Speed U* = {fd_flutter_speed:.4f}")

    plt.xlabel("Reduced Velocity (U*)")
    plt.ylabel("Damping Ratio")
    plt.title("Frequency Domain Flutter Analysis")
    plt.grid(True)
    plt.legend()

    plt.tight_layout()
    plt.savefig(os.path.join(result_dir, "fd_damping_analysis.png"), dpi=300)

    # 2. Frequency vs. velocity plot
    plt.figure(figsize=(10, 6))

    mode_1_freq = np.array([np.mean(fd_results[v]['frequency'][:, 0]) for v in velocity_range])
    mode_2_freq = np.array([np.mean(fd_results[v]['frequency'][:, 1]) for v in velocity_range])

    plt.plot(velocity_range, mode_1_freq, 'bo-', linewidth=2, label="Mode 1 Frequency")
    plt.plot(velocity_range, mode_2_freq, 'rs-', linewidth=2, label="Mode 2 Frequency")

    # Mark flutter speed if detected
    if fd_flutter_speed is not None:
        fd_flutter_speed = float(fd_flutter_speed)
        plt.axvline(x=fd_flutter_speed, color='g', linestyle='-', linewidth=2,
                   label=f"Flutter Speed U* = {fd_flutter_speed:.4f}")

    plt.xlabel("Reduced Velocity (U*)")
    plt.ylabel("Frequency")
    plt.title("Mode Frequencies vs. Velocity")
    plt.grid(True)
    plt.legend()

    plt.tight_layout()
    plt.savefig(os.path.join(result_dir, "fd_frequency_analysis.png"), dpi=300)

    # 3. Root locus plots for selected velocities
    if len(velocity_range) >= 3:
        # Choose 3 representative velocities
        if fd_flutter_speed is not None:
            fd_flutter_speed = float(fd_flutter_speed)
            v_below = max([v for v in velocity_range if v < fd_flutter_speed])
            v_near = min([v for v in velocity_range if v >= fd_flutter_speed])
            v_above = max([v for v in velocity_range if v > fd_flutter_speed * 1.1]) if any(v > fd_flutter_speed * 1.1 for v in velocity_range) else max(velocity_range)
            selected = [v_below, v_near, v_above]
        else:
            # If no flutter, pick first, middle, last
            idx = [0, len(velocity_range)//2, -1]
            selected = [velocity_range[i] for i in idx]

        fig, axs = plt.subplots(1, 3, figsize=(15, 5))

        for i, v in enumerate(selected):
            data = fd_results[v]

            # Get eigenvalues
            evals = data['eigenvalues']

            # Plot eigenvalues in complex plane
            for j in range(len(data['k_range'])):
                for k in range(4):
                    axs[i].plot(np.real(evals[j, k]), np.imag(evals[j, k]), 'k.', markersize=2)

            # Plot imaginary axis
            axs[i].axvline(x=0, color='k', linestyle='--', alpha=0.3)

            axs[i].set_title(f"Root Locus at U* = {float(v):.2f}")
            axs[i].set_xlabel("Real Part")
            axs[i].set_ylabel("Imaginary Part")
            axs[i].grid(True)

        plt.tight_layout()
        plt.savefig(os.path.join(result_dir, "fd_root_locus.png"), dpi=300)

ModuleNotFoundError: No module named 'flutter_model'

In [8]:
"""
2-DOF Aeroelastic Flutter Simulation - Main Production Script
-------------------------------------------------------------
This script runs a comprehensive production-quality aeroelastic flutter
simulation using physics-informed neural networks.
"""

import os
import time
import numpy as np
import matplotlib.pyplot as plt
import pickle

def run_production_flutter_simulation(param_set="ClassicalSection",
                                     velocity_mode="auto",
                                     specific_velocities=None,
                                     refinement=True,
                                     iterations=5000,
                                     time_domain=(0, 30),
                                     compare_with_frequency_domain=True):
    """
    Run a comprehensive production flutter simulation

    Parameters:
    ----------
    param_set : str
        Parameter set to use: "ClassicalSection", "LightAircraft", "NonConventional"
    velocity_mode : str
        How to choose velocities: "auto", "range", "specific"
    specific_velocities : list or None
        Specific velocities to analyze (if mode is "specific")
    refinement : bool
        Whether to perform refinement around flutter boundary
    iterations : int
        Number of training iterations per velocity
    time_domain : tuple
        (t_min, t_max) for the time domain
    compare_with_frequency_domain : bool
        Whether to run frequency domain analysis for comparison

    Returns:
    -------
    results : dict
        Results from the flutter analysis
    flutter_speed : float
        Estimated flutter speed
    """
    # Create results directory
    result_dir = create_results_directory("flutter_production")

    print("="*80)
    print("2-DOF Aeroelastic Flutter Simulation - Production Version")
    print("="*80)
    print(f"Started: {time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Results directory: {result_dir}")

    # Check DeepXDE installation
    print("\nChecking DeepXDE installation...")
    if not test_deepxde_installation():
        print("ERROR: DeepXDE installation test failed. Please check your environment.")
        return None, None

    # Select parameter set
    print("\nSetting up parameters...")
    if param_set == "ClassicalSection":
        params = AeroParams.classical_section()
    elif param_set == "LightAircraft":
        params = AeroParams.light_aircraft()
    elif param_set == "NonConventional":
        params = AeroParams.non_conventional()
    else:
        print(f"Unknown parameter set '{param_set}'. Using ClassicalSection instead.")
        params = AeroParams.classical_section()

    print(params)

    # Estimate flutter speed for auto mode
    approx_flutter = params.estimate_flutter_speed()
    print(f"Approximate analytical flutter speed: U* ≈ {approx_flutter:.2f}")

    # Set up velocity range based on mode
    if velocity_mode == "specific" and specific_velocities:
        # Use user-specified velocities
        velocity_range = np.array(specific_velocities)
        print(f"Using {len(velocity_range)} specific velocities provided by user")
    elif velocity_mode == "range":
        # Use broad range from 1 to 10
        velocity_range = np.linspace(1.0, 10.0, 10)
        print(f"Using velocity range from U* = {velocity_range[0]:.1f} to {velocity_range[-1]:.1f}")
    else:
        # Auto mode: use estimate to create efficient range
        v_min = max(0.5, approx_flutter * 0.5)
        v_max = approx_flutter * 1.5

        # Create non-uniform velocity range with more points near estimated flutter speed
        v_below = np.linspace(v_min, approx_flutter * 0.8, 3)
        v_near = np.linspace(approx_flutter * 0.8, approx_flutter * 1.2, 5)
        v_above = np.linspace(approx_flutter * 1.2, v_max, 3)
        velocity_range = np.unique(np.concatenate([v_below, v_near, v_above]))

        print(f"Auto mode: Using {len(velocity_range)} velocities from U* = {velocity_range[0]:.2f} to {velocity_range[-1]:.2f}")

    # Run flutter boundary analysis
    print("\nStarting flutter boundary analysis...")
    results, flutter_speed = analyze_flutter_boundary(
        params=params,
        velocity_range=velocity_range,
        time_domain=time_domain,
        iterations=iterations,
        result_dir=result_dir
    )

    # Refine flutter boundary if requested and flutter was detected
    if refinement and flutter_speed:
        print("\nRefining flutter boundary...")
        refined_results, refined_flutter_speed = refine_flutter_boundary(
            params=params,
            initial_results=results,
            flutter_speed=flutter_speed,
            refinement_range=flutter_speed * 0.1,  # 10% range around flutter speed
            num_points=5,
            time_domain=time_domain,
            iterations=iterations * 1.5,  # More iterations for refinement
            result_dir=result_dir
        )

        # Update results with refined analysis
        results = refined_results
        flutter_speed = refined_flutter_speed

    # Compare with frequency domain analysis if requested
    if compare_with_frequency_domain:
        print("\nRunning frequency domain analysis for comparison...")
        fd_results, fd_flutter_speed = perform_frequency_domain_analysis(
            params=params,
            velocity_range=velocity_range,
            result_dir=result_dir
        )

        # Compare results
        if flutter_speed and fd_flutter_speed:
            difference = abs(flutter_speed - fd_flutter_speed) / fd_flutter_speed * 100
            print(f"\nComparison of results:")
            print(f"  PINN flutter speed: U* = {flutter_speed:.4f}")
            print(f"  FD flutter speed:   U* = {fd_flutter_speed:.4f}")
            print(f"  Difference:         {difference:.2f}%")

        # Save comparison
        comparison = {
            'pinn_flutter_speed': flutter_speed,
            'fd_flutter_speed': fd_flutter_speed,
            'params': params,
            'velocity_range': velocity_range
        }

        with open(os.path.join(result_dir, "method_comparison.pkl"), 'wb') as f:
            pickle.dump(comparison, f)

    # Create overall summary report
    _create_summary_report(
        result_dir=result_dir,
        params=params,
        flutter_speed=flutter_speed,
        fd_flutter_speed=fd_flutter_speed if compare_with_frequency_domain else None,
        velocity_range=velocity_range,
        approx_flutter=approx_flutter
    )

    print("\n" + "="*80)
    print("Flutter simulation completed!")
    if flutter_speed:
        print(f"Flutter speed: U* = {flutter_speed:.4f}")
    else:
        print("No flutter detected in the analyzed velocity range")
    print(f"All results saved to: {result_dir}")
    print("="*80)

    return results, flutter_speed

def _create_summary_report(result_dir, params, flutter_speed, fd_flutter_speed,
                          velocity_range, approx_flutter):
    """Create a summary report of the analysis"""
    report_file = os.path.join(result_dir, "summary_report.txt")

    # Convert any numpy values to Python floats
    if flutter_speed is not None:
        flutter_speed = float(flutter_speed)
    if fd_flutter_speed is not None:
        fd_flutter_speed = float(fd_flutter_speed)
    approx_flutter = float(approx_flutter)
    velocity_range = [float(v) for v in velocity_range]

    with open(report_file, 'w') as f:
        f.write("="*80 + "\n")
        f.write("2-DOF AEROELASTIC FLUTTER SIMULATION - SUMMARY REPORT\n")
        f.write("="*80 + "\n\n")

        f.write(f"Date: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Results directory: {result_dir}\n\n")

        f.write("PARAMETERS:\n")
        f.write(f"  Parameter set: {params.name}\n")
        f.write(f"  Mass ratio (μ): {params.mu:.4f}\n")
        f.write(f"  Radius of gyration (r): {params.r:.4f}\n")
        f.write(f"  Center of mass (xθ): {params.x_theta:.4f}\n")
        f.write(f"  Elastic axis (a): {params.a:.4f}\n")
        f.write(f"  Stiffness (kh, kθ): {params.k_h:.4f}, {params.k_theta:.4f}\n")
        f.write(f"  Frequency ratio (ωh/ωθ): {params.frequency_ratio:.4f}\n\n")

        f.write("ANALYSIS DETAILS:\n")
        f.write(f"  Velocities analyzed: {len(velocity_range)}\n")
        f.write(f"  Velocity range: U* = {min(velocity_range):.2f} to {max(velocity_range):.2f}\n")
        f.write(f"  Analytical flutter estimate: U* ≈ {approx_flutter:.4f}\n\n")

        f.write("RESULTS:\n")
        if flutter_speed is not None:
            f.write(f"  PINN flutter speed: U* = {flutter_speed:.6f}\n")
        else:
            f.write("  No flutter detected with PINN method\n")

        if fd_flutter_speed is not None:
            f.write(f"  FD flutter speed:   U* = {fd_flutter_speed:.6f}\n")

            if flutter_speed is not None:
                difference = abs(flutter_speed - fd_flutter_speed) / fd_flutter_speed * 100
                f.write(f"  Difference:         {difference:.2f}%\n")

        f.write("\nNOTES:\n")
        f.write("  The flutter speed represents the velocity at which the\n")
        f.write("  aeroelastic system transitions from stable to unstable behavior.\n")
        f.write("  Above this speed, oscillations grow in amplitude over time.\n\n")

        f.write("  For a comprehensive understanding, examine the detailed plots\n")
        f.write("  in the results directory which show time histories, phase portraits,\n")
        f.write("  and stability metrics across the analyzed velocity range.\n")

    print(f"Summary report written to: {report_file}")

def compare_parameter_sets(param_sets=None, velocities=None, iterations=3000):
    """
    Compare flutter behavior across multiple parameter sets

    Parameters:
    ----------
    param_sets : list or None
        List of parameter set names to compare (default: all three)
    velocities : list or None
        Specific velocities to analyze (default: auto-determined)
    iterations : int
        Number of training iterations per velocity

    Returns:
    -------
    comparison_results : dict
        Comparison results for different parameter sets
    """
    # Default parameter sets if not specified
    if param_sets is None:
        param_sets = ["ClassicalSection", "LightAircraft", "NonConventional"]

    # Create results directory
    result_dir = create_results_directory("flutter_comparison")

    print("="*80)
    print("2-DOF Aeroelastic Flutter Parameter Comparison")
    print("="*80)

    # Initialize results
    comparison_results = {}

    # For each parameter set
    for param_set in param_sets:
        print(f"\n{'-'*60}")
        print(f"Analyzing parameter set: {param_set}")
        print(f"{'-'*60}")

        # Run simulation
        param_dir = os.path.join(result_dir, param_set)
        os.makedirs(param_dir, exist_ok=True)

        results, flutter_speed = run_production_flutter_simulation(
            param_set=param_set,
            velocity_mode="auto" if velocities is None else "specific",
            specific_velocities=velocities,
            refinement=True,
            iterations=iterations,
            time_domain=(0, 20),  # Shorter time domain for comparison
            compare_with_frequency_domain=True
        )

        # Store results
        comparison_results[param_set] = {
            'results': results,
            'flutter_speed': flutter_speed
        }

    # Create comparison visualizations
    _create_comparison_plots(comparison_results, result_dir)

    # Save comparison results
    with open(os.path.join(result_dir, "parameter_comparison.pkl"), 'wb') as f:
        pickle.dump(comparison_results, f)

    print("\n" + "="*80)
    print("Parameter comparison completed!")
    print(f"All comparison results saved to: {result_dir}")
    print("="*80)

    return comparison_results

def _create_comparison_plots(comparison_results, result_dir):
    """Create comparison plots across parameter sets"""
    param_sets = list(comparison_results.keys())

    # Flutter speed comparison
    flutter_speeds = []
    labels = []

    for param_set in param_sets:
        flutter_speed = comparison_results[param_set]['flutter_speed']
        if flutter_speed is not None:
            # Convert to Python float if numpy value
            flutter_speed = float(flutter_speed)
            flutter_speeds.append(flutter_speed)
            labels.append(param_set)

    if flutter_speeds:
        plt.figure(figsize=(10, 6))
        plt.bar(labels, flutter_speeds, color=['blue', 'green', 'red', 'orange', 'purple'][:len(labels)])

        for i, v in enumerate(flutter_speeds):
            plt.text(i, v + 0.1, f"{v:.2f}", ha='center')

        plt.xlabel("Parameter Set")
        plt.ylabel("Flutter Speed (U*)")
        plt.title("Flutter Speed Comparison Across Parameter Sets")
        plt.tight_layout()
        plt.savefig(os.path.join(result_dir, "flutter_speed_comparison.png"), dpi=300)



In [9]:
if __name__ == "__main__":
    # Example usage: run a production flutter simulation with default settings
    run_production_flutter_simulation(
        param_set="ClassicalSection",  # Use classical aeroelastic section parameters
        velocity_mode="auto",          # Automatically determine velocities
        refinement=True,               # Refine around flutter boundary
        iterations=5000,               # 5000 iterations per velocity
        time_domain=(0, 30),           # 0 to 30 time units
        compare_with_frequency_domain=True  # Compare with classical method
    )

    # For parameter comparison, uncomment:
    # compare_parameter_sets(iterations=3000)

2-DOF Aeroelastic Flutter Simulation - Production Version
Started: 2025-04-26 23:10:24
Results directory: flutter_production_20250426_231024

Checking DeepXDE installation...
Testing DeepXDE installation...
Compiling model...
Building feed-forward neural network...
'build' took 0.046041 s

'compile' took 0.337164 s

Training model...

Step      Train loss                        Test loss                         Test metric
0         [4.08e-01, 1.00e+00, 9.07e-01]    [4.08e-01, 1.00e+00, 9.07e-01]    []  
100       [8.50e-02, 6.38e-02, 7.91e-03]    [8.50e-02, 6.38e-02, 7.91e-03]    []  

Best model at step 100:
  train loss: 1.57e-01
  test loss: 1.57e-01
  test metric: []

'train' took 2.803589 s

Test completed with mean error: 0.161559
ERROR: DeepXDE installation test failed. Please check your environment.
